# 차량 추천

`langgraph-supervisor`는 LangChain에서 최근 발표된 LangGraph를 활용하여 **계층형 멀티 에이전트 시스템**을 구축하기 위한 Python 라이브러리입니다.
중앙의 **슈퍼바이저 에이전트**가 각 전문 에이전트를 총괄하며, **작업 할당**과 **통신 관리**를 담당합니다.
이를 통해 복잡한 작업을 효율적으로 처리할 수 있는 유연한 시스템을 구축할 수 있습니다.

**주요 기능**
- Supervisor Agent 생성:여러 명의 전문 에이전트를 총괄하며, 전체 작업 흐름을 조율합니다.
- 툴 기반의 에이전트 간 Handoff 메커니즘:에이전트 간 원활한 통신을 위한 전환(handoff) 메커니즘을 제공합니다.
- 유연한 메시지 기록 관리:대화의 흐름을 쉽게 제어할 수 있도록, 메시지 기록을 유연하게 관리할 수 있습니다.

## 1. 환경 설정

In [2]:
%pip install -qU langchain langgraph langchain-openai langgraph-supervisor

Note: you may need to restart the kernel to use updated packages.


In [4]:
# 환경 변수 확인
from dotenv import load_dotenv
import os
load_dotenv(override=True)

# TAVILY_API_KEY= os.environ.get("TAVILY_API_KEY")
# print(TAVILY_API_KEY[:20])

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("TAVILY_API_KEY")
_set_env("OPENAI_API_KEY")

## 2. LLM 객체 생성

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini")

## 3. 도구 생성

In [ ]:
car_information_path = "/xxx/xxx/車両情報.csv"

In [ ]:
# CSV 파일에서 차량 정보를 불러와 후보를 생성하는 예
def get_car_features():
    """The python code to get car information."""
    path = car_information_path
    df = pd.read_csv(car_information_path)
    car_features = df[["CAR_NAME", "BODY_TYPE", "DESCRIPTION"]].drop_duplicates()
    return car_features.to_dict(orient="records")

# 선택된 차종에 대해 엔진 타입을 추출하는 예
def get_engine_type(selected_car):
    """The python code to get engine type and engine information."""
    path = car_information_path
    df = pd.read_csv(car_information_path)
    engine_types = list(df[df["CAR_NAME"] == selected_car]["ENGINE_TYPE"].unique())
    return engine_types, "ENGINE_DETAIL"

## 4. 에이전트 생성

In [ ]:
from langgraph.prebuilt import create_react_agent

# 차량 추천 에이전트
car_agent = create_react_agent(
    model=llm,
    tools=[get_car_features],
    name="car_agent",
    prompt="""
    # 지시문
    제안 패턴에 따라 추천할 차량 종류를 선택하고, 그 근거를 약 200자 정도로 설명해주세요.
    """
)

# 엔진타입 선택 에이전트 
engine_agent = create_react_agent(
    model=llm,
    tools=[get_engine_type],
    name="engine_agent",
    prompt="""
    # 지시문
    추천 차량에 가장 적합한 엔진 타입을 선택하고, 그 근거를 약 200자 정도로 설명해주세요.
    """
)

## 5. 슈퍼바이저 에이전트

각 에이전트를 총괄하는 Supervisor를 생성하고, 고객 정보를 바탕으로 최종 제안을 생성합니다.
여기서의 핵심 아이디어는 Supervisor에 입력되는 프롬프트를 모듈화하여, 높은 재사용성과 범용성을 확보한 것입니다.
role이나 task 변수의 내용과 연결된 에이전트를 바꾸기만 하면, 다른 작업에도 쉽게 응용할 수 있습니

In [ ]:
from langgraph_supervisor import create_supervisor

# 프롬프트의 각 모듈 내용을 작성
role = "자동차 판매원"
task = "고객 정보를 바탕으로, 최적의 자동차와 엔진 타입을 제안해 주세요."
guideline = """
    - 출력은 반드시 최종 생성불 포맷에 맞추어서 JSON 형식으로 해주세요.
    - 각 근거는 200자 정도의 풍부한 문장으로 작성해 주세요.
    """
output_format = """
    {
        "제안 내용": {
            "제안할 차종",
            "차종 선택의 근거",
            "선택한 엔진 타입",
            "엔진 선택 이유"
        }
    }
    """

# 프롬프트 작성
system_prompt = f"""
    ## 역할
    당신은 뛰어난 {role}입니다.

    ## 작업 
    {task}

    ## 가이드라인
    {guideline}
    
    ## 최종 생성물 포맷
    {output_format}
    """

# Supervisor 생성
workflow = create_supervisor(
    # 앞서 생성한 에이전트와 연결
    [car_agent, engine_agent],
    model=llm,
    prompt=system_prompt
)

# 그래프 컴파일
app = workflow.compile()


## 6. 실행

In [ ]:
from langchain.schema import HumanMessage
import time

# 고객 정보 예시 (필요에 따라 상세 정보를 추가)
customer_info = "고객 정보: 나이 35세, 연비 중시, 현재 탑승 중인 차량은 소형차"


# 실행
start_time = time.time()
result = app.invoke({
    "messages": [
        {"role": "user", "content": customer_info}
    ]
})
end_time = time.time()

# 최종 출력 표시
print("최종 제안:")
print(result["messages"][-1].content)
print("실행 시간: {:.3f}초".format(end_time - start_time))


In [ ]:
# 최종 제안:
# {
#     "제안 내용": {
#         "제안하는 차종": "하이브리드 기술을 적용한 최신 컴팩트카",
#         "차종의 근거": "고객은 현재 컴팩트카를 운전 중이며, 연비 성능을 중시하고 있습니다. 하이브리드 기술을 적용한 차량은 연료 사용 효율이 높고, 경제적 부담과 환경에 대한 배려도 우수합니다. 또한, 하이브리드 시스템은 단거리나 도심 주행에도 적합합니다. 따라서 최신 하이브리드 컴팩트카가 최적의 선택지입니다.",
#         "선택한 엔진 타입": "하이브리드 엔진",
#         "엔진 선택 이유": "하이브리드 엔진은 연비 성능이 매우 우수하며, 고객의 니즈인 연료 효율을 최우선으로 고려했습니다. 게다가 컴팩트카와의 궁합도 좋고, 도심 이용이나 일상 이동에서 높은 퍼포먼스를 발휘합니다. 이 엔진의 선택은 쾌적성, 경제성, 환경 성능을 모두 충족합니다."
#     }
# }
# 실행 시간: 21.938초
